<a href="https://colab.research.google.com/github/coder272377/dvp/blob/feeder/yf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import functools
import getpass
import os
import urllib.parse as up

from typing import Dict, Union



In [ ]:
import pandas as pd
import requests



## Managing packages

In [ ]:
try:
  import mplfinance as fplt
except ModuleNotFoundError:
  !pip install mplfinance
  import mplfinance as fplt



## Getting the data



* https://github.com/ranaroussi/yfinance/blob/main/yfinance/base.py



In [ ]:
user_agent_headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}



In [ ]:
def chart_url(ticker:str) -> str:
  """
  Get json URL for the ticker
  """
  return (
      "https://query2.finance.yahoo.com"
      f"/v8/finance/chart/{ticker}"
  )



In [ ]:
def get_request_param(
    start:pd.Timestamp, end:pd.Timestamp, interval:str,
    prepost:bool=False,
  ) -> Dict[str, Union[int, str, bool]]:
  return {
      "period1": int(start.timestamp()),
      "period2": int(end.timestamp()),
      "interval": interval,
      "includePrePost": prepost,
  }



In [ ]:
def get_index_ny(json) -> pd.Series:
  data = []

  for t in json["chart"]["result"][0]["timestamp"]:
    timestamp = pd.Timestamp(t, unit='s', tz="GMT").tz_convert(tz_ny())
    data.append(timestamp)
  
  return pd.Series(data)



In [ ]:
@functools.lru_cache()
def tz_ny():
  return "America/New_York"



In [ ]:
def get_df_start_end(
    ticker:str, start:pd.Timestamp, end:pd.Timestamp, interval:str="1m"
  ) -> pd.DataFrame:
  """
  Get data for n days
  """

  params=get_request_param(start, end, interval)

  r = requests.get(
      url=chart_url(ticker),
      params=params,
      headers=user_agent_headers,
  )

  assert r.ok, r.text

  json = r.json()

  return pd.DataFrame(
      data=json["chart"]["result"][0]["indicators"]["quote"][0],
      index=get_index_ny(json),
  )



In [ ]:
end = pd.Timestamp.today(tz=tz_ny())
start = end + pd.Timedelta(-5, 'D')

df = get_df_start_end("SPY", start, end, "5m")



In [ ]:
assert df.index[0].date() <= start.date(), f"first index = {df.index[0]} start = {start}"
assert df.index[-1].date() <= end.date(), f"last index = {df.index[-1]} end = {end}"



In [ ]:
fplt.plot(
  df,
  type='candle',
  style='charles',
  ylabel='Price ($)',
  datetime_format="%y/%m/%d %H:%M"
)

